# Lab 11: Grid Localization using Bayes Filter (Virtual Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

<hr>


In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
from localization_extras import Localization

# Setup Logger
LOG = get_logger('demo_notebook.log')

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

gui.show()

# Start the simulator
START_SIM()

# Start the plotter
START_PLOTTER()

2025-04-23 09:22:54,026 | INFO     |: Logger demo_notebook.log initialized.


TwoByTwoLayout(children=(Label(value='Simulator', layout=Layout(grid_area='top-left', width='80px')), HBox(chi…

In [2]:
# Initialize Robot to communicate with the virtual robot and plotter
robot = VirtualRobot(cmdr)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a VirtualRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2025-04-23 09:23:16,870 | INFO     |:  | Number of observations per grid cell: 18
2025-04-23 09:23:16,872 | INFO     |:  | Precaching Views...


C:\Users\qijin\Documents\Ece4160\IvannnHuang.github.io\Lab11\localization.py:150: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(distance_intersections_tt), intersections_tt[np.nanargmin(distance_intersections_tt)]


2025-04-23 09:23:21,474 | INFO     |:  | Precaching Time: 4.601 secs
2025-04-23 09:23:21,475 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-04-23 09:23:21,476 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


# Run the Bayes Filter
The cells below utilizes the member functions of class **Localization** (defined in [localization_extras.py](../localization_extras.py)) in each iteration of the Bayes filter algorithm to localize the robot in the grid map. <br>

In [3]:
# Reset Robot and Plots
robot.reset()
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.print_update_stats(plot_data=True)

# Plot Odom and GT
current_odom, current_gt = robot.get_pose()
cmdr.plot_gt(current_gt[0], current_gt[1])
cmdr.plot_odom(current_odom[0], current_odom[1])

2025-04-23 09:23:29,359 | INFO     |: Initializing beliefs with a Uniform Distribution
2025-04-23 09:23:29,360 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
2025-04-23 09:23:32,305 | INFO     |: Update Step
2025-04-23 09:23:32,307 | INFO     |:      | Update Time: 0.001 secs
2025-04-23 09:23:32,308 | INFO     |: ---------- UPDATE STATS -----------
2025-04-23 09:23:32,320 | INFO     |: GT index      : (6, 4, 9)
2025-04-23 09:23:32,321 | INFO     |: Bel index     : (np.int64(5), np.int64(4), np.int64(9)) with prob = 0.6923920
2025-04-23 09:23:32,322 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2025-04-23 09:23:32,323 | INFO     |: GT            : (0.000, 0.000, 360.000)
2025-04-23 09:23:32,324 | INFO     |: Belief        : (0.000, 0.000, 10.000)
2025-04-23 09:23:32,324 | INFO     |: POS ERROR     : (-0.000, -0.000, 350.000)
2025-04-23 09:23:32,325 | INFO     |: ---------- UPDATE STATS -----------


In [4]:
# Initialize the Trajectory object
traj = Trajectory(loc)

# Run through each motion steps
for t in range(0, traj.total_time_steps):
    print("\n\n-----------------", t, "-----------------")
    
    prev_odom, current_odom, prev_gt, current_gt = traj.execute_time_step(t)
        
    # Prediction Step
    loc.prediction_step(current_odom, prev_odom)
    loc.print_prediction_stats(plot_data=True)
    
    # Get Observation Data by executing a 360 degree rotation motion
    loc.get_observation_data()
    
    # Update Step
    loc.update_step()
    loc.print_update_stats(plot_data=True)

# Uncomment the below line to wait for keyboard input between each iteration.
#   input("Press Enter to Continue")
        
    print("-------------------------------------")



----------------- 0 -----------------
2025-04-23 09:24:01,155 | INFO     |: Prediction Step
2025-04-23 09:24:01,275 | INFO     |:  | Prediction Time: 0.119 secs
2025-04-23 09:24:01,276 | INFO     |: ---------- PREDICTION STATS -----------
2025-04-23 09:24:01,289 | INFO     |: GT index         : (6, 3, 6)
2025-04-23 09:24:01,290 | INFO     |: Prior Bel index  : (np.int64(7), np.int64(2), np.int64(9)) with prob = 0.0674091
2025-04-23 09:24:01,291 | INFO     |: POS ERROR        : (-0.328, 0.522, -50.107)
2025-04-23 09:24:01,293 | INFO     |: ---------- PREDICTION STATS -----------
2025-04-23 09:24:04,261 | INFO     |: Update Step
2025-04-23 09:24:04,263 | INFO     |:      | Update Time: 0.001 secs
2025-04-23 09:24:04,275 | INFO     |: ---------- UPDATE STATS -----------
2025-04-23 09:24:04,291 | INFO     |: GT index      : (6, 3, 6)
2025-04-23 09:24:04,292 | INFO     |: Bel index     : (np.int64(6), np.int64(4), np.int64(6)) with prob = 0.9999999
2025-04-23 09:24:04,293 | INFO     |: Be